# Hyperasの練習

*20170715 Mitsuhiro Ito*


* (https://github.com/maxpumperla/hyperas/tree/master/examples)


---

In [1]:
from __future__ import print_function
import sys
from utils import save_model_viz, save_weights, save_hist, plot_hist
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import uniform, choice, conditional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [3]:
def data():
    train_gan = ImageDataGenerator(rescale=1.0 / 255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    valid_gan = ImageDataGenerator(rescale=1.0 / 255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    train_generator = train_gan.flow_from_directory('/home/dl/mitsu-work/data/train/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')

    valid_generator = valid_gan.flow_from_directory('/home/dl/mitsu-work/data/valid/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')
    
    
    
    return train_generator, valid_generator

In [4]:
def model(train_generator, valid_generator):

    model = Sequential()
    model.add(Conv2D({{choice([32, 64, 128])}}, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D({{choice([32, 64, 128])}}, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D({{choice([32, 64, 128])}}, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense({{choice([32, 64, 128])}}))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    batch_size = {{choice([16, 32])}}


    model.fit_generator(train_generator, 
                    steps_per_epoch = 2000 // batch_size, 
                    epochs=25, 
                    verbose=1, 
                    validation_data = valid_generator,
                    validation_steps= 800 // batch_size)
    
    
    score, acc = model.evaluate_generator(generator=valid_generator, 
                                      steps= 800 // batch_size)
    
    print(score, acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [5]:
train_generator, valid_generator = data()

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=10,
                                      trials=Trials(),
                                      notebook_name='20170714_hyperas4' )
print("Evalutation of best performing model:")
print(best_model.evaluate_generator(valid_generator, steps=64))
print(best_run)



Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import sys
except:
    pass

try:
    from utils import save_model_viz, save_weights, save_hist, plot_hist
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import uniform, choice, conditional
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.layers.convolutional import Convolution2D, MaxPooling2D
except:
    pass

try:
    from keras.optimizers import SGD
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try

125/125 [==============================] - 6s - loss: 0.0340 - acc: 0.9880 - val_loss: 1.6209 - val_acc: 0.7075
Epoch 24/25
125/125 [==============================] - 6s - loss: 0.0353 - acc: 0.9870 - val_loss: 1.4840 - val_acc: 0.6775
Epoch 25/25
125/125 [==============================] - 6s - loss: 0.0158 - acc: 0.9960 - val_loss: 1.8162 - val_acc: 0.6775
1.64116328657 0.71125
Epoch 1/25
125/125 [==============================] - 7s - loss: 0.6962 - acc: 0.5220 - val_loss: 0.6933 - val_acc: 0.4888
Epoch 2/25
125/125 [==============================] - 6s - loss: 0.6947 - acc: 0.5035 - val_loss: 0.6925 - val_acc: 0.5150
Epoch 3/25
125/125 [==============================] - 7s - loss: 0.6934 - acc: 0.5055 - val_loss: 0.6934 - val_acc: 0.4900
Epoch 4/25
125/125 [==============================] - 7s - loss: 0.6930 - acc: 0.5075 - val_loss: 0.6926 - val_acc: 0.5025
Epoch 5/25
125/125 [==============================] - 7s - loss: 0.6921 - acc: 0.5240 - val_loss: 0.6910 - val_acc: 0.5238
Epo

62/62 [==============================] - 3s - loss: 0.6918 - acc: 0.4980 - val_loss: 0.6931 - val_acc: 0.5075
Epoch 15/25
62/62 [==============================] - 3s - loss: 0.6919 - acc: 0.5202 - val_loss: 0.6935 - val_acc: 0.4800
Epoch 16/25
62/62 [==============================] - 3s - loss: 0.6938 - acc: 0.5071 - val_loss: 0.6930 - val_acc: 0.5125
Epoch 17/25
62/62 [==============================] - 3s - loss: 0.6935 - acc: 0.5040 - val_loss: 0.6932 - val_acc: 0.4950
Epoch 18/25
62/62 [==============================] - 3s - loss: 0.6940 - acc: 0.4738 - val_loss: 0.6931 - val_acc: 0.5175
Epoch 19/25
62/62 [==============================] - 3s - loss: 0.6928 - acc: 0.4940 - val_loss: 0.6932 - val_acc: 0.4850
Epoch 20/25
62/62 [==============================] - 3s - loss: 0.6926 - acc: 0.5050 - val_loss: 0.6932 - val_acc: 0.4950
Epoch 21/25
62/62 [==============================] - 3s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.4825
Epoch 22/25
62/62 [=================

62/62 [==============================] - 3s - loss: 0.6933 - acc: 0.5192 - val_loss: 0.6933 - val_acc: 0.4775
Epoch 6/25
62/62 [==============================] - 3s - loss: 0.6933 - acc: 0.4980 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 7/25
62/62 [==============================] - 3s - loss: 0.6935 - acc: 0.4849 - val_loss: 0.6949 - val_acc: 0.4800
Epoch 8/25
62/62 [==============================] - 3s - loss: 0.6934 - acc: 0.5161 - val_loss: 0.6925 - val_acc: 0.5450
Epoch 9/25
62/62 [==============================] - 3s - loss: 0.6910 - acc: 0.5333 - val_loss: 0.6890 - val_acc: 0.5750
Epoch 10/25
62/62 [==============================] - 3s - loss: 0.6924 - acc: 0.5091 - val_loss: 0.6926 - val_acc: 0.5000
Epoch 11/25
62/62 [==============================] - 3s - loss: 0.6947 - acc: 0.4829 - val_loss: 0.6918 - val_acc: 0.5150
Epoch 12/25
62/62 [==============================] - 2s - loss: 0.6904 - acc: 0.5323 - val_loss: 0.6836 - val_acc: 0.5425
Epoch 13/25
62/62 [=====================

125/125 [==============================] - 7s - loss: 0.0190 - acc: 0.9940 - val_loss: 1.7288 - val_acc: 0.6825
Epoch 22/25
125/125 [==============================] - 7s - loss: 0.0206 - acc: 0.9945 - val_loss: 1.9923 - val_acc: 0.6550
Epoch 23/25
125/125 [==============================] - 7s - loss: 0.0105 - acc: 0.9980 - val_loss: 1.9445 - val_acc: 0.6825
Epoch 24/25
125/125 [==============================] - 7s - loss: 0.0191 - acc: 0.9940 - val_loss: 1.9706 - val_acc: 0.6713
Epoch 25/25
125/125 [==============================] - 6s - loss: 0.0072 - acc: 0.9980 - val_loss: 2.3854 - val_acc: 0.6375
2.25852489829 0.65375
Epoch 1/25
62/62 [==============================] - 3s - loss: 0.6961 - acc: 0.4859 - val_loss: 0.6922 - val_acc: 0.5350
Epoch 2/25
62/62 [==============================] - 3s - loss: 0.6933 - acc: 0.5181 - val_loss: 0.6823 - val_acc: 0.6225
Epoch 3/25
62/62 [==============================] - 3s - loss: 0.6935 - acc: 0.5665 - val_loss: 0.6721 - val_acc: 0.5775
Epoch 4